In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
import os
import seaborn as sns


# set the legend frame
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.edgecolor'] = 'black'
plt.rcParams['legend.framealpha'] = 1

# set the line width
plt.rcParams['lines.linewidth'] = 2.0

# set the point size
plt.rcParams['lines.markersize'] = 8

# set the error bar capsize
plt.rcParams['errorbar.capsize'] = 5

# set the font size
plt.rcParams['font.size'] = 16

# set the figure dpi
plt.rcParams['figure.dpi'] = 192

# set the save figure format
plt.rcParams['savefig.format'] = 'pdf'

sns.set_style("whitegrid")
sns.set_style("ticks")
sns.set_context("paper")


In [2]:
targets = ["llvm", "opencl", "cuda"]
networks = ["mlp", "mlp_conv", "separable_conv1d", "separable_conv2d", "mlp_wide", "mlp_deep", "mlp_deep_wide", "conv1d", "conv2d", "depthwise_conv1d", "lstm", "mlp_piecewise", "mlp_piecewise2", "mlp_big", "mlp_small"]
old_cluster = 7476309
version = "0.0.0"
old_cluster2 = 7473664

if version[2] == "0":
    windowsize = 3
else:
    windowsize = 4

In [3]:
best = []
fpga = []
for i, network in enumerate(networks):
    fig , ax = plt.subplots(figsize=(10, 6))
    for j, target in enumerate(targets):
        files = glob.glob(f'../../final/benchmarks/{network}-*{version}-{target}-{old_cluster}*d.perf')
        if files == []:
            best.append(np.nan)
            continue
        #files = glob.glob(f'../../final/benchmarks/{network}-*-{target}-{old_cluster}*d.perf')
        #print(files)
        batch_size = []
        data = []
        for file in files:
            data.append(np.loadtxt(file, delimiter=','))
            #print(f"Loaded {file}")
            batch_size.append(int(re.findall(r'-\d+', file)[0][1:]))
            #print(f"Batch size {batch_size[-1]}")
        me = np.array([np.mean(d) for d in data])
        st = np.array([np.std(d) for d in data])
        batch_size = np.array(batch_size)
        ax.plot(batch_size, me/batch_size*1e6, ['s', 'o', '*'][j], label=target)
        # Make best of each target in a box and place it at 10 and bachsize of 4096
        ax.text(4096, np.exp(j*0.35+1), fr"{target}: {np.amin(me/batch_size*1e6):.2f} $\mu$ s", ha='center', va='center', bbox=dict(facecolor='white', edgecolor='black', alpha=1))
        best.append(np.amin(me/batch_size*1e6))
        print(f"Best {np.amin(me/batch_size*1e6)} for {network} {target}")
    if os.path.exists(f"hls4ml/{network}_{version}_{old_cluster2}_hls4ml_prj/myproject_prj/solution1/syn/report/myproject_csynth.rpt"):
        hls4ml_rpt = f"hls4ml/{network}_{version}_{old_cluster2}_hls4ml_prj/myproject_prj/solution1/syn/report/myproject_csynth.rpt"
        # Find the max latency
        with open(hls4ml_rpt) as f:
            for j, line in enumerate(f):
                if "+ Latency:" in line:
                    # Get the latency 5 lines down
                    print(f"Found latency in {hls4ml_rpt}")
                    for _ in range(6):
                        line = next(f)
                    latency = re.findall(r'\d+', line)[-1]
                    ax.plot([0,65565], [int(latency)*5e-9*1e6, int(latency)*5e-9*1e6], 'r', label="FPGA")
                    ax.text(4096, np.exp(3*0.35+1), fr"FPGA: {int(latency)*5e-9*1e6:.2f} $\mu$ s", ha='center', va='center', bbox=dict(facecolor='white', edgecolor='black', alpha=1))
                    fpga.append(int(latency)*5e-9*1e6)
    else:
        fpga.append(np.nan)
    #ax.plot(batch_size, np.ones_like(batch_size)*44*5e-9, 'r', label="FPGA")
    title = f"{network} performance for window size of {windowsize}" + (" inside-out" if version[-1] == "0" else " outside-in") +  (" with float32" if old_cluster == 7476308 else " with float16")
    ax.set_title(title)
    ax.set_xlim(0.6, 50000)
    ax.legend()
    ax.set_xlabel("Batch size")
    ax.set_ylabel(r"Time per sample ($\mu$s)")
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.grid(True)
    plt.tight_layout()
    #plt.show()
    if old_cluster == 7476308:
        plt.savefig(f"../../final/runtime/{network}_runtime_{version}.pdf")
    else:
        plt.savefig(f"../../final/runtime/{network}_runtime_{version}_fp16.pdf")
    plt.close()



Best 0.1532872783238921 for mlp llvm
Best 0.08942271321614582 for mlp opencl
Best 0.08782820476955837 for mlp cuda
Found latency in hls4ml/mlp_0.0.0_7473664_hls4ml_prj/myproject_prj/solution1/syn/report/myproject_csynth.rpt
Best 0.6020073207855224 for mlp_conv llvm
Best 0.13429278218068802 for mlp_conv opencl
Best 0.12966323141665878 for mlp_conv cuda
Found latency in hls4ml/mlp_conv_0.0.0_7473664_hls4ml_prj/myproject_prj/solution1/syn/report/myproject_csynth.rpt
Best 0.27195650590931536 for separable_conv1d llvm
Best 0.10451301635608011 for separable_conv1d opencl
Best 0.10440406438528439 for separable_conv1d cuda
Best 0.3787093506167633 for separable_conv2d llvm
Best 0.10423617994805345 for separable_conv2d opencl
Best 0.09983665528407851 for separable_conv2d cuda
Best 0.6190702731045811 for mlp_wide llvm
Best 0.34331463989257816 for mlp_wide opencl
Best 0.34136137008666995 for mlp_wide cuda
Found latency in hls4ml/mlp_wide_0.0.0_7473664_hls4ml_prj/myproject_prj/solution1/syn/report/

In [ ]:
#make a bar plot of the stds using seaborn
import pandas as pd
data = []
for i, network in enumerate(networks):
    for j, target in enumerate(targets):
        data.append([network, target, best[i*3+j]])
    if not np.isnan(fpga[i]):
        data.append([network, "FPGA", fpga[i]])
df = pd.DataFrame(data, columns=['Network', 'Target', 'Time'])
fig, ax = plt.subplots(1, figsize=(18, 6))
sns.barplot(df, x="Network", y="Time", ax=ax, linewidth=1.5, edgecolor="black", hue="Target")
ax.set_title(f"Best performance per network for window size of {windowsize}"+ (" inside-out" if version[2] == "0" else " outside-in") + (" with float32" if old_cluster == 7476308 else " with float16"), fontsize=20)
ax.set_xlabel("Network", fontsize=18)
ax.set_ylabel(r"Time per sample ($\mu$s)", fontsize=18)
ax.set_xticklabels(networks, rotation=45, ha="right")
ax.grid()
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.1f'), 
                (p.get_x() + p.get_width() / 2., np.amax([np.amin([p.get_height(),0.95]),0.1])), 
                ha = 'center', va = 'center', 
                xytext = (0, -10), 
                textcoords = 'offset points',
                color='white', rotation=90)
plt.tight_layout()
# set the size of the labels
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=14)
sns.despine(top=True, right=False)
ax.set_ylim(0,1)
#plt.show()
if old_cluster == 7476308:
    plt.savefig(f"../../final/runtime/best_performance_{version}.pdf", bbox_inches='tight')
else:
    plt.savefig(f"../../final/runtime/best_performance_{version}_fp16.pdf", bbox_inches='tight')
plt.close()




/tmp/ipykernel_272793/1980051621.py:15: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(networks, rotation=45, ha="right")
